In [7]:
from mainnet_launch.abis import CHAINLINK_KEEPER_REGISTRY_ABI
from mainnet_launch.constants import ALL_AUTOPOOLS, ETH_CHAIN

from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions_table_for_gas_costs import *
from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df, insert_avoid_conflicts
from mainnet_launch.database.schema.full import Transactions, Blocks, ChainlinkGasCosts

# TODO convert to TOKEMAK address with multichain

KEEPER_REGISTRY_CONTRACT_ADDRESS = "0x6593c7De001fC8542bB1703532EE1E5aA0D458fD"


def _fetch_topic_id_to_highest_already_fetched_block(chain: ChainData) -> dict[str, int]:
    # TODO this can be only in SQL
    prior_df = merge_tables_as_df(
        [
            TableSelector(ChainlinkGasCosts, select_fields=[ChainlinkGasCosts.chainlink_topic_id]),
            TableSelector(
                Transactions,
                select_fields=[Transactions.tx_hash],
                join_on=ChainlinkGasCosts.tx_hash == Transactions.tx_hash,
            ),
            TableSelector(Blocks, select_fields=Blocks.datetime, join_on=Transactions.block == Blocks.block),
        ],
        where_clause=Blocks.chain_id == chain.chain_id,
    )
    topic_id_to_highest_already_fetched_block = prior_df.groupby("chainlink_topic_id")["block"].max().to_dict()
    return topic_id_to_highest_already_fetched_block


def _ensure_one_chain_chainlink_gas_costs_is_updated(
    chain: ChainData, chainlink_keepers_df: pd.DataFrame
) -> pd.DataFrame:

    topic_id_to_highest_already_fetched_block = _fetch_topic_id_to_highest_already_fetched_block(chain)

    keeper_network_topic_ids_to_name = (
        chainlink_keepers_df[chainlink_keepers_df["chain_id"] == chain.chain_id].set_index("id")["name"].to_dict()
    )
    upkeep_dfs = []

    contract = chain.client.eth.contract(KEEPER_REGISTRY_CONTRACT_ADDRESS, abi=CHAINLINK_KEEPER_REGISTRY_ABI)

    for topic_id, name in keeper_network_topic_ids_to_name.items():
        highest_block_already_fetched = topic_id_to_highest_already_fetched_block.get(topic_id, 0)
        # note still fetches for deprecated blocks, but it is fine
        # TODO somehow handle `deprecated` topic ids
        # not really needed

        # seperate fetch for each topic id
        # because the blocks can be different and otherwise we duplicate fetches
        # or miss some blocks
        our_upkeep_df = fetch_events(
            contract.events.UpkeepPerformed,
            chain=chain,
            start_block=highest_block_already_fetched,
            argument_filters={"id": int(topic_id)},
        )
        our_upkeep_df["name"] = name
        upkeep_dfs.append(our_upkeep_df)

    full_upkeep_df = pd.concat(upkeep_dfs, ignore_index=True)
    if full_upkeep_df.empty:
        # early exit if no new events
        return

    ensure_all_transactions_are_saved_in_db(full_upkeep_df["hash"].unique().tolist(), chain)

    new_chainlink_gas_costs_rows = full_upkeep_df.apply(
        lambda row: ChainlinkGasCosts(
            tx_hash=row["hash"],
            chainlink_topic_id=row["id"],
        ),
        axis=1,
    ).tolist()

    insert_avoid_conflicts(
        ChainlinkGasCosts,
        new_chainlink_gas_costs_rows,
    )


def ensure_chainlink_gas_costs_table_is_updated() -> None:
    deployers_df, chainlink_keepers_df, service_accounts_df = fetch_systems_df()
    for chain in [ETH_CHAIN]:
        _ensure_one_chain_chainlink_gas_costs_is_updated(chain, chainlink_keepers_df)


ensure_chainlink_gas_costs_table_is_updated()

KeyError: 'Column not found: block'

In [ ]:
break

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions_table_for_gas_costs import *

deployers_df, chainlink_keepers_df, service_accounts_df = fetch_systems_df()

In [ ]:
keeper_network_topic_ids_to_name = (
    chainlink_keepers_df[chainlink_keepers_df["chain_id"] == 1].set_index("id")["name"].to_dict()
)

keeper_network_topic_ids_to_name

{'84910810589923801598536031507827941923735631663622593132512932471876788938876': 'Incentive Pricing',
 '2774403708484311544165440706031341871504925629391958533428545305548524420937': 'Eth Per Token Sender',
 '1344461886831441856282597505993515040672606510446374000438363195934269203116': 'Calculators',
 '113129673265054907567420460651277872997162644350081440026681710279139531871240': 'Calculators',
 '93443706906332180407535184303815616290343141548650473059299738217546322242910': 'Calculators'}

In [ ]:
from mainnet_launch.abis import CHAINLINK_KEEPER_REGISTRY_ABI
from mainnet_launch.constants import ALL_AUTOPOOLS, ETH_CHAIN


from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df
from mainnet_launch.database.schema.full import Transactions, Blocks, ChainlinkGasCosts


KEEPER_REGISTRY_CONTRACT_ADDRESS = "0x6593c7De001fC8542bB1703532EE1E5aA0D458fD"

prior_df = merge_tables_as_df(
    [
        TableSelector(ChainlinkGasCosts, select_fields=[ChainlinkGasCosts.chainlink_topic_id]),
        TableSelector(Transactions, join_on=ChainlinkGasCosts.tx_hash == Transactions.tx_hash),
        TableSelector(Blocks, select_fields=Blocks.datetime, join_on=Transactions.block == Blocks.block),
    ],
    where_clause=Blocks.chain_id == 1,
)

prior_df

,chainlink_topic_id,tx_hash,block,chain_id,from_address,to_address,effective_gas_price,gas_used,gas_cost_in_eth,datetime


In [ ]:
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
)

# ensure_all_transactions_are_saved_in_db(transaction_hashes_required, chain)

topiuc_id_to_highest_block_already_seen = prior_df.groupby("chainlink_topic_id")["block"].max().to_dict()

upkeep_dfs = []

contract = ETH_CHAIN.client.eth.contract(KEEPER_REGISTRY_CONTRACT_ADDRESS, abi=CHAINLINK_KEEPER_REGISTRY_ABI)

for topic_id, name in keeper_network_topic_ids_to_name.items():
    highest_block_already_fetched = topiuc_id_to_highest_block_already_seen.get(topic_id, 0)
    # note still fetches for deprecated blocks, but it is fine
    # TODO somehow handle `deprecated` topic ids

    our_upkeep_df = fetch_events(
        contract.events.UpkeepPerformed,
        chain=ETH_CHAIN,
        start_block=highest_block_already_fetched,
        argument_filters={"id": int(topic_id)},
    )
    upkeep_dfs.append(our_upkeep_df)

In [ ]:
full_upkeep_df = pd.concat(upkeep_dfs, ignore_index=True)
full_upkeep_df

,event,block,transaction_index,log_index,hash,id,success,totalPayment,gasUsed,gasOverhead,trigger
0,UpkeepPerformed,20645297,75,241,0x0933d4b5b108d7d8195a12b9154c890bac33485c78d3...,8491081058992380159853603150782794192373563166...,True,113582692139007409,215745,114066,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00..."
1,UpkeepPerformed,20645300,23,115,0x5be559dec57a71d2bee78dd0333d84e993163d5e85ae...,8491081058992380159853603150782794192373563166...,True,89596749770120566,123614,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
2,UpkeepPerformed,20645302,28,130,0x04714742d28a6b7ab6694f17b755b54e988e238f201f...,8491081058992380159853603150782794192373563166...,True,69806807857322891,79112,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
3,UpkeepPerformed,20645303,10,54,0x514f65d1b34c74be2ec05689e3692adeec5cc1bbdb1e...,8491081058992380159853603150782794192373563166...,True,98001822279915588,153733,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
4,UpkeepPerformed,20645306,19,155,0x613084f9ae5906ae30cc9c914bf4d2ecd4f1abe5fbd3...,8491081058992380159853603150782794192373563166...,True,82689517229467215,103863,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...,...,...,...,...,...
19129,UpkeepPerformed,22919069,2,34,0x793509f1a8a319ad70b0ccc2e0fc4f0491e6602912d3...,9344370690633218040753518430381561629034314154...,True,3545188142851469624,1571873,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
19130,UpkeepPerformed,22919364,23,168,0x312e836417a91a59f78b2ffc5bdcd0de1f9e6d1392ca...,9344370690633218040753518430381561629034314154...,True,3880427797310145348,1386177,112718,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00..."
19131,UpkeepPerformed,22919367,82,379,0xe3e119f20b55b07437705cc225f176116bb0c5a4a558...,9344370690633218040753518430381561629034314154...,True,1878874545269029070,552011,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
19132,UpkeepPerformed,22919380,104,415,0xced5f6af774ab7a4ff9fda7bfcc9295f2713f3b0fdb8...,9344370690633218040753518430381561629034314154...,True,4429473778540582433,1273527,112718,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...


In [ ]:
ensure_all_transactions_are_saved_in_db(full_upkeep_df["hash"].unique().tolist(), ETH_CHAIN)
# 20k chainlink keeper transactions takes 10s minutes to get and save

KeyboardInterrupt: 

,event,block,transaction_index,log_index,hash,id,success,totalPayment,gasUsed,gasOverhead,trigger
0,UpkeepPerformed,20645297,75,241,0x0933d4b5b108d7d8195a12b9154c890bac33485c78d3...,8491081058992380159853603150782794192373563166...,True,113582692139007409,215745,114066,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00..."
1,UpkeepPerformed,20645300,23,115,0x5be559dec57a71d2bee78dd0333d84e993163d5e85ae...,8491081058992380159853603150782794192373563166...,True,89596749770120566,123614,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
2,UpkeepPerformed,20645302,28,130,0x04714742d28a6b7ab6694f17b755b54e988e238f201f...,8491081058992380159853603150782794192373563166...,True,69806807857322891,79112,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
3,UpkeepPerformed,20645303,10,54,0x514f65d1b34c74be2ec05689e3692adeec5cc1bbdb1e...,8491081058992380159853603150782794192373563166...,True,98001822279915588,153733,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
4,UpkeepPerformed,20645306,19,155,0x613084f9ae5906ae30cc9c914bf4d2ecd4f1abe5fbd3...,8491081058992380159853603150782794192373563166...,True,82689517229467215,103863,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...,...,...,...,...,...
19127,UpkeepPerformed,22919069,2,34,0x793509f1a8a319ad70b0ccc2e0fc4f0491e6602912d3...,9344370690633218040753518430381561629034314154...,True,3545188142851469624,1571873,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
19128,UpkeepPerformed,22919364,23,168,0x312e836417a91a59f78b2ffc5bdcd0de1f9e6d1392ca...,9344370690633218040753518430381561629034314154...,True,3880427797310145348,1386177,112718,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00..."
19129,UpkeepPerformed,22919367,82,379,0xe3e119f20b55b07437705cc225f176116bb0c5a4a558...,9344370690633218040753518430381561629034314154...,True,1878874545269029070,552011,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
19130,UpkeepPerformed,22919380,104,415,0xced5f6af774ab7a4ff9fda7bfcc9295f2713f3b0fdb8...,9344370690633218040753518430381561629034314154...,True,4429473778540582433,1273527,112718,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...


2025-07-14 16:23:02,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-14 16:23:02,341 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-07-14 16:23:02,342 INFO sqlalchemy.engine.Engine [cached since 3127s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x324472140>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-07-14 16:23:02,342 INFO sqlalchemy.engine.Engine SELECT
    chainlink_gas_costs.chainlink_topic_id,
    transactions.tx_hash,
    blocks.datetime
FROM chainlink_gas_costs
JOIN t

KeyError: 'Column not found: block'